### Implementation with autograd

In [45]:
import autograd.numpy as anp
import numpy as np
from autograd import jacobian

np.set_printoptions(suppress=True)


def newton_autograd(F, x0, delta, m):
    J = jacobian(F)
    x = np.asarray(x0, dtype=float)

    for i in range(m):
        s = np.linalg.solve(J(x), -F(x))
        x = x + s

        if np.linalg.norm(F(x)) < delta:
            return {
                "solution": x,
                "iterations": i + 1,
                "message": f"Convergence reached after {i+1} iterations",
            }

    return {
        "solution": x,
        "iterations": m,
        "message": f"Maximum iterations ({m}) reached",
    }


if __name__ == "__main__":
    L1 = 1
    L2 = 0.8

    xd = 0.5
    yd = 0.8

    F = lambda x: anp.array(
        [
            L2 * anp.cos(x[0] + x[1]) + L1 * anp.cos(x[0]) - xd,
            L2 * anp.sin(x[0] + x[1]) + L1 * anp.sin(x[0]) - yd,
        ]
    )

    x0 = [1, 1]
    delta = 1e-5
    m = 1000

    r = newton_autograd(F, x0, delta, m)

    print(f"Solution: θ1 = {r['solution'][0]:.6f}, θ2 = {r['solution'][1]:.6f}")
    print(f"Iterations: {r['iterations']}")

Solution: θ1 = 0.165550, θ2 = 2.058671
Iterations: 5


### Implementation calculating the Jacobian manually

In [ ]:
def newton_manual(F, J, x0, delta, m):
    x = np.asarray(x0, dtype=float)

    for i in range(m):
        s = np.linalg.solve(J(x), -F(x))
        x = x + s

        if np.linalg.norm(F(x)) < delta:
            return {
                "solution": x,
                "iterations": i + 1,
                "message": f"Convergence reached after {i+1} iterations",
            }

    return {
        "solution": x,
        "iterations": m,
        "message": f"Maximum iterations ({m}) reached",
    }


if __name__ == "__main__":
    F = lambda x: np.array(
        [
            L2 * np.cos(x[0] + x[1]) + L1 * np.cos(x[0]) - xd,
            L2 * np.sin(x[0] + x[1]) + L1 * np.sin(x[0]) - yd,
        ]
    )
    
    # Manual Jacobian matrix
    def J(x):
        θ1, θ2 = x[0], x[1]
        
        # partial derivatives of F        
        dF1_dtheta1 = -L2 * np.sin(θ1 + θ2) - L1 * np.sin(θ1)
        dF1_dtheta2 = -L2 * np.sin(θ1 + θ2)
        dF2_dtheta1 = L2 * np.cos(θ1 + θ2) + L1 * np.cos(θ1)
        dF2_dtheta2 = L2 * np.cos(θ1 + θ2)
        
        return np.array([[dF1_dtheta1, dF1_dtheta2],
                         [dF2_dtheta1, dF2_dtheta2]])

    r = newton_manual(F, J, x0, delta, m)

    print(f"Solution: θ1 = {r['solution'][0]:.6f}, θ2 = {r['solution'][1]:.6f}")
    print(f"Iterations: {r['iterations']}")

Solution: θ1 = 0.165550, θ2 = 2.058671
Iterations: 5


In [47]:
import time

F_autograd = lambda x: anp.array(
        [
            L2 * anp.cos(x[0] + x[1]) + L1 * anp.cos(x[0]) - xd,
            L2 * anp.sin(x[0] + x[1]) + L1 * anp.sin(x[0]) - yd,
        ]
    )

F_manual = lambda x: np.array(
    [
        L2 * np.cos(x[0] + x[1]) + L1 * np.cos(x[0]) - xd,
        L2 * np.sin(x[0] + x[1]) + L1 * np.sin(x[0]) - yd,
    ]
)

def J_manual(x):
    t1, t2 = x[0], x[1]
    dF1_dtheta1 = -L2 * np.sin(t1 + t2) - L1 * np.sin(t1)
    dF1_dtheta2 = -L2 * np.sin(t1 + t2)
    dF2_dtheta1 = L2 * np.cos(t1 + t2) + L1 * np.cos(t1)
    dF2_dtheta2 = L2 * np.cos(t1 + t2)
    return np.array([[dF1_dtheta1, dF1_dtheta2],
                     [dF2_dtheta1, dF2_dtheta2]])

# Autograd
start_time = time.time()
result_autograd = newton_autograd(F_autograd, x0, delta, m)
time_autograd = time.time() - start_time

# Manual Jacobian calculation
start_time = time.time()  
result_manual = newton_manual(F_manual, J_manual, x0, delta, m)
time_manual = time.time() - start_time

print(f"Autograd Approach:")
print(f"  Solution: θ₁ = {result_autograd['solution'][0]:.8f}, θ₂ = {result_autograd['solution'][1]:.8f}")
print(f"  Iterations: {result_autograd['iterations']}")
print(f"  Time: {time_autograd*1000:.4f} ms")
print()
print(f"Manual Jacobian Approach:")
print(f"  Solution: θ₁ = {result_manual['solution'][0]:.8f}, θ₂ = {result_manual['solution'][1]:.8f}")
print(f"  Iterations: {result_manual['iterations']}")
print(f"  Time: {time_manual*1000:.4f} ms")

Autograd Approach:
  Solution: θ₁ = 0.16555028, θ₂ = 2.05867148
  Iterations: 5
  Time: 4.2090 ms

Manual Jacobian Approach:
  Solution: θ₁ = 0.16555028, θ₂ = 2.05867148
  Iterations: 5
  Time: 0.4499 ms


### Least squares to solve for the step

In [48]:
def newton_autograd_lstsq(F, x0, delta, m):
    J = jacobian(F)
    x = np.asarray(x0, dtype=float)

    for i in range(m):
        s, _, _, _ = np.linalg.lstsq(J(x), -F(x), rcond=None)
        x = x + s

        if np.linalg.norm(F(x)) < delta:
            return {
                "solution": x,
                "message": f"Convergence reached after {i+1} iterations",
            }

    return {
        "solution": x,
        "message": f"Maximum iterations ({m}) reached",
    }
    
if __name__ == "__main__":
    L3 = 0.6
    
    F = lambda x: anp.array([
        L1*anp.cos(x[0]) + L2*anp.cos(x[0] + x[1]) + L3*anp.cos(x[0] + x[1] + x[2]) - xd,
        L1*anp.sin(x[0]) + L2*anp.sin(x[0] + x[1]) + L3*anp.cos(x[0] + x[1] + x[2]) - yd
    ])

    # inital theta (in radians)
    x0 = [0.6, 0.5, 0.9] 

    result = newton_autograd_lstsq(F, x0, delta=1e-8, m=100)
    print(f"Solution: {result['solution']}")
    print(f"Message: {result['message']}")


Solution: [0.56265731 0.79471677 1.2463618 ]
Message: Convergence reached after 4 iterations
